In [1]:
import random, numpy as np, pandas as pd
from matplotlib import pyplot as plt
alphap = 3
probp = 0.75
expon = 2 * alphap - 1
slope = 0.78835

In [2]:
def betadist(alpha):
    """gives a random number from beta distribution"""
    return random.betavariate(alpha,alpha)

In [3]:
def decision(probability):
    """
    decides with a given probability whether to keep the right part
    """
    if float(probability) > random.random():
        return True
    else: 
        return False

In [4]:
def splitting(segment):
    """
    splits a given segment. left and right are endpoints of the segment
    returns : 
        xL -> length of the left segment
        xR -> length of the right segment
        flag -> keeping the right segment
        xLp, xRp -> probability(unnormalized) for being selected
        change -> change of normalization const
    """
    xL = segment * betadist(alphap)
    xR = segment - xL
    flag = decision(probp)
    xLp = xL**expon
    xRp = xR**expon
    change = xLp + xRp - segment**expon
    return xL, xR, flag, xLp, xRp, change

In [5]:
# def pickindex(frag_prob, frag_prob_sum):
#     """
#     picks up a segment to be subsequently split
#     """
#     r = random.uniform(0, frag_prob_sum)
#     sum_ = 0
#     for index in range(len(frag_prob)):
#         sum_ += frag_prob[index]
#         if sum_ < r:
#             continue
#         else:
#             return index

In [6]:
def pickindex(frag_prob, frac_sum):
    """
    picks up a segment to be subsequently split
    """
    r = random.uniform(0,1)
    
    if r <= frac_sum:
        sum_ = 0
        for index in range(len(frag_prob)):
            sum_ += frag_prob[index]
            if sum_ < r:
                continue
            else:
                return index

In [7]:
def number_length(segment_lengths,flags):
    
    N = 0
    M = 0

    for i in range(len(flags)):
        if flags[i]:
            N += 1
            M += segment_lengths[i]
            pass
        pass
    return N, M

In [8]:
fractal_dim = slope

def fractal_length(segment_lengths,flags):

    M_frac = 0

    for i in range(len(flags)):
        if flags[i]:
            M_frac += segment_lengths[i]**fractal_dim
            pass
        pass
    return M_frac

In [9]:
def realization_value(total_iteration, min_iteration, iteration_step, data_point = False):
    
    lengths = [1.]
    flags = [True]
    frag_prob = [1.] # raw probability, not normalized
    #frag_prob_sum = 1.0 # normalization const
    frac_sum = 1.0

    iteration_list = list(range(min_iteration, total_iteration + 1, iteration_step))
    
    M_realization = []
    
    for i in range(total_iteration + 1):
        
        #index = pickindex(frag_prob, frag_prob_sum)
        index = pickindex(frag_prob, frac_sum)
        
        if index != None and flags[index] == True:

            xL, xR, flag, xLp, xRp, change = splitting(lengths[index])
            
            lengths[index] = xL
            lengths.append(xR)
            flags.append(flag)
            frag_prob[index] = xLp 
            frag_prob.append(xRp)
            #frag_prob_sum += change
            frac_sum += change
            pass
        
        if i+1 in iteration_list:
            M_frac = fractal_length(lengths,flags)
            M_realization.append(M_frac)
        pass 
      
    if data_point:
        print(np.array(M_realization))
    
    else:
        M_frac = fractal_length(lengths,flags)
        return M_frac

In [10]:
def ensemble_average(total_iteration, ensemble_size):

    M_ensemble = 0
    
    for i in range(ensemble_size):
        M_list = realization_value(total_iteration, total_iteration, total_iteration)
        M_ensemble += M_list
        pass
    
    M_average = M_ensemble/ensemble_size
    
    print(M_average)

In [11]:
for i in range(5):
    realization_value(1_000_000, 100_000, 100_000, data_point = True)

[1.44000382 1.46343067 1.36066612 1.37260029 1.37260029 1.38915808
 1.40262208 1.41982952 1.42819757 1.44290759]
[1.19142756 1.10585628 1.14706226 1.17183572 1.19581243 1.17148413
 1.17148413 1.17148413 1.17148413 1.18428835]
[0.75992694 0.75992694 0.75992694 0.7766056  0.78817294 0.78817294
 0.78817294 0.78817294 0.78817294 0.78817294]
[0.64440922 0.66113306 0.66113306 0.55578377 0.55578377 0.55578377
 0.55578377 0.55578377 0.55578377 0.55578377]
[0.32762479 0.32762479 0.32762479 0.32762479 0.32762479 0.32762479
 0.32762479 0.32762479 0.32762479 0.32762479]


In [12]:
for i in range(10):
    a_ = 1_000*(i+1)
    ensemble_average(a_, 100_000)

1.0010604014581141
1.0004598538062768
1.0001982849347757
0.9984938404965412
0.9971459460161585
1.0006431267256037
0.9995080218311788
0.9998727720245041
0.9983210795452307
1.0014042928618103
